**Configuración dual de credenciales - Ahora hay dos conjuntos de credenciales, uno para leer tweets (monitorear las cuentas) y otro para publicar tus señales contrarias (pueden ser los mismos si vas a usar una sola cuenta).
Función mejorada para generar señales - La función generate_contrary_signal ahora crea un mensaje formateado para Twitter que incluye:**

La señal contraria (COMPRAR/VENDER)
El mercado y/o símbolos relevantes
Hashtags para mayor visibilidad
Referencia a la cuenta original
Timestamp para indicar cuándo se generó la señal


Publicación automática en Twitter - Nueva función post_to_twitter que publica la señal contraria como un tweet, opcionalmente en respuesta al tweet original.
Seguimiento de publicaciones - Ahora el script registra qué señales se han publicado y guarda los IDs de los tweets para evitar duplicados.
Verificación de inicio - Al iniciar, el script verifica que las credenciales para publicar sean correctas.

**Para usar este script, necesitarás:**

Registrar una aplicación de desarrollador en la plataforma de Twitter/X para obtener las credenciales API (las actuales y más recientes)
Llenar tus credenciales en las variables correspondientes
Asegurarte que tu cuenta tenga los permisos necesarios (lectura y escritura)
Especificar las cuentas que quieres monitorear en la lista target_accounts

In [ ]:
import tweepy
import time
import re
import pandas as pd
from datetime import datetime, timedelta
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Descargar recursos necesarios de NLTK
nltk.download('vader_lexicon')

# Configuración de las credenciales de API de Twitter (X)
# Credenciales para leer tweets
consumer_key = "TU_CONSUMER_KEY"
consumer_secret = "TU_CONSUMER_SECRET"
access_token = "TU_ACCESS_TOKEN"
access_token_secret = "TU_ACCESS_TOKEN_SECRET"

# Autenticación con Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Credenciales para la cuenta que publicará las señales (puede ser la misma)
posting_consumer_key = "TU_POSTING_CONSUMER_KEY"
posting_consumer_secret = "TU_POSTING_CONSUMER_SECRET"
posting_access_token = "TU_POSTING_ACCESS_TOKEN"
posting_access_token_secret = "TU_POSTING_ACCESS_TOKEN_SECRET"

# Autenticación para la cuenta que publicará
posting_auth = tweepy.OAuthHandler(posting_consumer_key, posting_consumer_secret)
posting_auth.set_access_token(posting_access_token, posting_access_token_secret)
posting_api = tweepy.API(posting_auth, wait_on_rate_limit=True)

# Configurar las cuentas a monitorear
target_accounts = ["cuenta1", "cuenta2"]

# Palabras clave para detección de mercados
keywords_argentina = ["merval", "argentina", "peso", "ars", "cedear", "bcra", "dólar", "dolar",
                     "bonos", "leliq", "acción argentina", "bolsa argentina", "galicia", "ypf"]

keywords_internacional = ["sp500", "nasdaq", "dow jones", "nyse", "forex", "fed", "Wall Street",
                         "treasury", "usd", "eur", "oil", "commodities", "bitcoin", "crypto"]

# Palabras clave para señales alcistas/bajistas
bullish_words = ["comprar", "largo", "alcista", "subir", "bull", "bullish", "long", "upside", "buy",
                "sube", "rebote", "oportunidad", "crecimiento", "soporte"]

bearish_words = ["vender", "corto", "bajista", "caer", "bear", "bearish", "short", "downside", "sell",
                "baja", "cae", "resistencia", "corrección", "sobrecomprado"]

# Inicializar analizador de sentimiento
sid = SentimentIntensityAnalyzer()

# Función para determinar el mercado mencionado
def detect_market(text):
    text_lower = text.lower()
    argentina_score = sum(1 for word in keywords_argentina if word.lower() in text_lower)
    internacional_score = sum(1 for word in keywords_internacional if word.lower() in text_lower)

    if argentina_score > internacional_score:
        return "argentino"
    elif internacional_score > argentina_score:
        return "internacional"
    elif argentina_score > 0 and internacional_score > 0:
        return "ambos"
    else:
        return "indeterminado"

# Función para determinar si el tweet es alcista o bajista
def detect_sentiment(text):
    text_lower = text.lower()

    # Contar palabras alcistas y bajistas
    bullish_count = sum(1 for word in bullish_words if word.lower() in text_lower)
    bearish_count = sum(1 for word in bearish_words if word.lower() in text_lower)

    # Usar análisis de sentimiento para complementar
    sentiment_score = sid.polarity_scores(text)['compound']

    # Ponderación combinada
    if bullish_count > bearish_count and sentiment_score > 0:
        return "alcista"
    elif bearish_count > bullish_count and sentiment_score < 0:
        return "bajista"
    elif bullish_count > bearish_count:
        return "posiblemente alcista"
    elif bearish_count > bullish_count:
        return "posiblemente bajista"
    elif sentiment_score > 0.3:
        return "sentimiento alcista"
    elif sentiment_score < -0.3:
        return "sentimiento bajista"
    else:
        return "neutral"

# Función para extraer símbolos de acciones mencionados en el tweet
def extract_symbols(text):
    # Buscar patrones como $AAPL, $YPF, etc.
    symbols = re.findall(r'\$([A-Za-z0-9]+)', text)
    return symbols if symbols else ["general"]

# Función para generar la señal contraria
def generate_contrary_signal(sentiment, symbols, market, account):
    if "alcista" in sentiment:
        action = "VENDER"
    elif "bajista" in sentiment:
        action = "COMPRAR"
    else:
        return None  # Sin señal clara

    symbols_text = ', '.join(symbols)
    market_text = f"mercado {market}" if "general" in symbols else symbols_text

    # Crear mensaje para publicar
    message = f"🔄 SEÑAL CONTRARIA a @{account}: {action} {market_text}\n"
    message += f"#{market.upper()} #{action} "

    # Agregar hashtags de los símbolos
    for symbol in symbols:
        if symbol != "general":
            message += f"#{symbol} "

    # Añadir timestamp
    now = datetime.now().strftime("%H:%M:%S")
    message += f"\n⏰ {now}"

    return message

# Función para publicar en Twitter
def post_to_twitter(message, original_tweet_id=None):
    try:
        # Publica tweet, opcionalmente como respuesta al original
        if original_tweet_id:
            status = posting_api.update_status(
                status=message,
                in_reply_to_status_id=original_tweet_id,
                auto_populate_reply_metadata=True
            )
        else:
            status = posting_api.update_status(status=message)

        print(f"Tweet publicado con éxito: {status.id}")
        return status.id
    except Exception as e:
        print(f"Error al publicar en Twitter: {e}")
        return None

# Función para verificar si ya se generó una señal para un tweet específico
def already_processed(tweet_id, results_df):
    return results_df['original_tweet_id'].astype(str).isin([str(tweet_id)]).any()

# Función principal para monitorear tweets
def monitor_tweets():
    print("Iniciando monitoreo de tweets...")

    # Crear dataframe para guardar resultados
    columns = ["fecha", "cuenta", "tweet", "original_tweet_id", "mercado", "sentimiento",
              "símbolos", "señal_contraria", "señal_publicada", "señal_tweet_id"]
    results_df = pd.DataFrame(columns=columns)

    # Cargar resultados anteriores si existen
    try:
        existing_df = pd.read_csv("señales_mercado.csv")
        results_df = existing_df
        print(f"Cargados {len(existing_df)} registros previos.")
    except:
        print("No se encontraron registros previos. Iniciando nuevo registro.")

    while True:
        try:
            for account in target_accounts:
                # Obtener los últimos tweets
                tweets = api.user_timeline(screen_name=account,
                                          count=10,
                                          include_rts=False,
                                          tweet_mode='extended')

                # Procesar cada tweet
                for tweet in tweets:
                    # Solo procesar tweets de las últimas 24 horas
                    tweet_time = tweet.created_at
                    if datetime.now() - tweet_time > timedelta(hours=24):
                        continue

                    # Verificar si este tweet ya fue procesado
                    if already_processed(tweet.id, results_df):
                        continue

                    # Obtener texto completo
                    text = tweet.full_text

                    # Analizar el tweet
                    market = detect_market(text)
                    sentiment = detect_sentiment(text)
                    symbols = extract_symbols(text)

                    # Solo procesar si es sobre mercados y tiene sentimiento definido
                    if market != "indeterminado" and sentiment != "neutral":
                        # Generar señal contraria
                        signal_message = generate_contrary_signal(sentiment, symbols, market, account)

                        if signal_message:
                            # Publicar la señal en Twitter
                            signal_tweet_id = post_to_twitter(signal_message, tweet.id)

                            # Agregar a resultados
                            new_row = {
                                "fecha": tweet_time,
                                "cuenta": account,
                                "tweet": text,
                                "original_tweet_id": tweet.id,
                                "mercado": market,
                                "sentimiento": sentiment,
                                "símbolos": ",".join(symbols),
                                "señal_contraria": signal_message,
                                "señal_publicada": True if signal_tweet_id else False,
                                "señal_tweet_id": signal_tweet_id if signal_tweet_id else None
                            }

                            results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)

                            # Mostrar la señal
                            print(f"\n{'='*50}")
                            print(f"Cuenta: @{account}")
                            print(f"Tweet: {text}")
                            print(f"Mercado: {market}")
                            print(f"Sentimiento: {sentiment}")
                            print(f"Acciones mencionadas: {symbols}")
                            print(f"SEÑAL PUBLICADA: {signal_message}")
                            if signal_tweet_id:
                                print(f"ID del tweet publicado: {signal_tweet_id}")
                            print(f"{'='*50}\n")

                            # Guardar resultados en CSV
                            results_df.to_csv("señales_mercado.csv", index=False)

            # Esperar 15 minutos antes de volver a verificar
            print(f"Verificación completada a las {datetime.now().strftime('%H:%M:%S')}")
            print("Esperando 15 minutos para la próxima verificación...")
            time.sleep(15 * 60)

        except Exception as e:
            print(f"Error: {e}")
            print("Reintentando en 5 minutos...")
            time.sleep(5 * 60)

if __name__ == "__main__":
    try:
        # Verificar que se puede publicar en Twitter
        test_status = posting_api.verify_credentials()
        print(f"Autenticación exitosa para publicar como: @{test_status.screen_name}")

        # Iniciar monitoreo
        monitor_tweets()
    except Exception as e:
        print(f"Error en la inicialización: {e}")